## Învățarea prin transfer folosind datele MNIST
Pentru a ilustra puterea și conceptul de învățare prin transfer, vom antrena o rețea neuronală convoluțională (CNN) doar pe cifrele 5,6,7,8,9. Apoi vom antrena doar ultimul strat (straturi) al rețelei pe cifrele 0,1,2,3,4 și vom vedea cât de bine ajută caracteristicile învățate pe 5-9 la clasificarea 0-4.

Adaptat de la https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py

In [1]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [2]:
# Folosit pentru a ajuta unele dintre funcțiile de temporizare
now = datetime.datetime.now

In [3]:
# setam unii parametri
batch_size = 128
num_classes = 5
epochs = 5

In [4]:
# setam mai multi parametri
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

In [5]:
## Acest lucru gestionează doar o anumită variabilitate în modul în care datele de intrare sunt încărcate.

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [6]:
## Pentru a simplifica lucrurile, scrieți o funcție pentru a include toți pașii de antrenament
## Ca intrare, funcția primește un model, setul de antrenament, setul de testare și numărul de clase
## În interiorul obiectului modelului va fi starea despre ce straturi înghețăm și ce straturi antrenăm

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [7]:
# datele, amestecate și împărțite între seturile de antrenare și testare
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# creați două seturi de date: unul cu cifre mai mici decât 5 și unul cu 5 și mai mari
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [8]:
# Definiți straturile "feature". Acestea sunt straturile timpurii pe care ne așteptăm să "transferăm"
# la o nouă problemă. Vom îngheța aceste straturi în timpul procesului de ajustare fino.

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [9]:
# Definiți straturile "classification". Acestea sunt straturile ulterioare care prezic clasele specifice din caracteristicile
# învățate de straturile "feature". Aceasta este partea modelului care trebuie re-antrenată pentru o problemă nouă

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [10]:
# Modelul nostru este creat prin combinarea celor două seturi de straturi astfel:
model = Sequential(feature_layers + classification_layers)

In [11]:
# Să aruncăm o privire
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [12]:
# Acum, să antrenăm modelul nostru pe cifrele 5,6,7,8,9

train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 47s 199ms/step - loss: 1.6114 - accuracy: 0.2056 - val_loss: 1.5950 - val_accuracy: 0.2907
Epoch 2/5
230/230 [==============================] - 45s 196ms/step - loss: 1.5936 - accuracy: 0.2511 - val_loss: 1.5758 - val_accuracy: 0.3843
Epoch 3/5
230/230 [==============================] - 53s 231ms/step - loss: 1.5750 - accuracy: 0.3065 - val_loss: 1.5558 - val_accuracy: 0.4950
Epoch 4/5
230/230 [==============================] - 50s 217ms/step - loss: 1.5560 - accuracy: 0.3645 - val_loss: 1.5340 - val_accuracy: 0.6135
Epoch 5/5
230/230 [==============================] - 48s 210ms/step - loss: 1.5353 - accuracy: 0.4264 - val_loss: 1.5096 - val_accuracy: 0.7056
Training time: 0:04:02.985329
Test score: 1.509598970413208
Test accuracy: 0.7056161165237427


### Înghețarea Straturilor
Keras permite "înghețarea" straturilor în timpul procesului de antrenament. Adică, unele straturi ar avea actualizate ponderile lor în timpul procesului de antrenament, în timp ce altele nu ar. Acest lucru este o parte de bază a învățării prin transfer, capacitatea de a antrena doar ultimul strat sau mai multe straturi.

De asemenea, rețineți că mult timpul de antrenament este petrecut "propagând" gradientele înapoi la primul strat. Prin urmare, dacă trebuie să calculăm gradientele înapoi doar pentru un număr mic de straturi, timpul de antrenament este mult mai rapid pe iterație. Acest lucru este în plus față de economiile obținute prin posibilitatea de a antrena pe un set de date mai mic.

In [13]:
# Înghețați doar 
for l in feature_layers:
    l.trainable = False

Observați diferențele dintre numărul total de *parametri*, *parametri antrenabili* și *parametri non-antrenabili*.

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [15]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 20s 81ms/step - loss: 1.5636 - accuracy: 0.3373 - val_loss: 1.5421 - val_accuracy: 0.4797
Epoch 2/5
240/240 [==============================] - 19s 79ms/step - loss: 1.5336 - accuracy: 0.4048 - val_loss: 1.5099 - val_accuracy: 0.5472
Epoch 3/5
240/240 [==============================] - 21s 86ms/step - loss: 1.5039 - accuracy: 0.4684 - val_loss: 1.4766 - val_accuracy: 0.6468
Epoch 4/5
240/240 [==============================] - 19s 81ms/step - loss: 1.4741 - accuracy: 0.5200 - val_loss: 1.4423 - val_accuracy: 0.7342
Epoch 5/5
240/240 [==============================] - 18s 76ms/step - loss: 1.4423 - accuracy: 0.5710 - val_loss: 1.4070 - val_accuracy: 0.7867
Training time: 0:01:37.401651
Test score: 1.407010555267334
Test accuracy: 0.7867289185523987


Rețineți că după o singură epocă, obținem deja rezultate în clasificarea cifrelor 0-4 care sunt comparabile cu cele obținute pentru cifrele 5-9 după 5 epoci complete. Aceasta în ciuda faptului că "ajustăm fin" doar ultimul strat al rețelei, iar toate straturile anterioare nu au văzut niciodată cum arată cifrele 0-4.

De asemenea, observați că, chiar dacă aproape toți (590K/600K) *parametrii* erau antrenabili, timpul de antrenament pe epocă a fost totuși mult redus. Aceasta se datorează faptului că partea neînghețată a rețelei era foarte puțin adâncă, făcând propagarea înapoi mai rapidă.

## Exercițiu
### De făcut:
- Acum scrieți cod pentru a inversa acest proces de antrenament. Adică, veți antrena pe cifrele 0-4, iar apoi veți ajusta fin doar ultimele straturi pe cifrele 5-9.

In [16]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train_0_4 = x_train[y_train < 5]
y_train_0_4 = y_train[y_train < 5]

x_train_5_9 = x_train[y_train >= 5]
y_train_5_9 = y_train[y_train >= 5] - 5

x_train_0_4 = x_train_0_4.astype('float32') / 255
x_train_5_9 = x_train_5_9.astype('float32') / 255

x_train_0_4 = x_train_0_4.reshape(x_train_0_4.shape[0], 28, 28, 1)
x_train_5_9 = x_train_5_9.reshape(x_train_5_9.shape[0], 28, 28, 1)

y_train_0_4 = to_categorical(y_train_0_4, 5)
y_train_5_9 = to_categorical(y_train_5_9, 5)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train_0_4, y_train_0_4, batch_size=128, epochs=5, verbose=1)

for layer in model.layers[:-2]:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train_5_9, y_train_5_9, batch_size=128, epochs=5, verbose=1)

Epoch 1/5
240/240 [==============================] - 28s 116ms/step - loss: 0.1302 - accuracy: 0.9630
Epoch 2/5
240/240 [==============================] - 24s 102ms/step - loss: 0.0257 - accuracy: 0.9923
Epoch 3/5
240/240 [==============================] - 25s 103ms/step - loss: 0.0165 - accuracy: 0.9948
Epoch 4/5
240/240 [==============================] - 26s 108ms/step - loss: 0.0122 - accuracy: 0.9963
Epoch 5/5
240/240 [==============================] - 29s 121ms/step - loss: 0.0079 - accuracy: 0.9977
Epoch 1/5
230/230 [==============================] - 14s 41ms/step - loss: 0.2994 - accuracy: 0.9295
Epoch 2/5
230/230 [==============================] - 9s 39ms/step - loss: 0.0496 - accuracy: 0.9844
Epoch 3/5
230/230 [==============================] - 7s 33ms/step - loss: 0.0345 - accuracy: 0.9891
Epoch 4/5
230/230 [==============================] - 9s 38ms/step - loss: 0.0258 - accuracy: 0.9924
Epoch 5/5
230/230 [==============================] - 8s 35ms/step - loss: 0.0225 - accura